<a href="https://colab.research.google.com/github/harshil0217/NFL_Rookie_Comps/blob/main/ProfileEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
from google.colab import auth
from google.cloud import storage
import io
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import pipeline
import torch
from torch.utils.data import Dataset, DataLoader
auth.authenticate_user()

In [2]:
client = storage.Client(project='cmse-381')
bucket = client.get_bucket('stats_draftprospects')
blob = bucket.blob('draft_profiles.csv')
content = blob.download_as_string()
profiles = pd.read_csv(io.BytesIO(content))

In [3]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,Brown has played second fiddle to Carnell Will...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,Williams started two games and played in nine ...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,Jefferson is an efficient back who follows and...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,Stuard is an undersized off-the-ball linebacke...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,Jones started games at guard at the junior co...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [8]:
#use BERT-NER to remove Person, Organization, Place, Date, and Time


device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

class ProfileDataset(Dataset):
    def __init__(self, profiles):
        self.profiles = profiles
        self.nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=device, batch_size=32, aggregation_strategy="simple")

    def __len__(self):
        return len(self.profiles)

    def __getitem__(self, idx):
        draft_profile = self.profiles.iloc[idx]['draft_profile']
        processed_profile = self.remove_entities(draft_profile)
        return processed_profile

    def remove_entities(self, text):
      entities = self.nlp(text)
      for entity in entities:
        text = text.replace(entity['word'], '')
      return text

dataset = ProfileDataset(profiles)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

processed_profiles = []
for batch in dataloader:
    processed_profiles.extend(batch)

profiles['draft_profile'] = processed_profiles

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [9]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,has played second fiddle to nell throughout ...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,was drafted by the in the th round out of hi...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,is a savvy route runner with average burst an...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,uard is an undersized off-the-ball linebacker ...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,started games at guard at the junior college...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [53]:
# add sentiment column with bert, using default bert tokenizer

tokenizer_sentiment = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased-finetuned-sst-2-english')
model_sentiment = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-uncased-finetuned-sst-2-english')
sentiment_pipeline = pipeline('sentiment-analysis', model=model_sentiment, tokenizer=tokenizer_sentiment, device = device)

#ge5 label
def get_sentiment(text):
    text = text[:512]
    sentiment = sentiment_pipeline(text)
    if sentiment[0]['label'] == 'NEGATIVE':
        return -1
    else:
        return str(sentiment[0]['score'])


profiles['sentiment'] = profiles['draft_profile'].apply(get_sentiment)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [54]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image,sentiment
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN,-1
1,Ronnie Brown,has played second fiddle to nell throughout ...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN,-1
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN,0.9977142810821533
3,Cedric Benson,was drafted by the in the th round out of hi...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN,0.9970932006835938
4,Carnell Williams,started two games and played in nine in befo...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,-1
4022,Dax Milne,is a savvy route runner with average burst an...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,0.9980408549308777
4023,Grant Stuard,uard is an undersized off-the-ball linebacker ...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...,0.9987921118736267
4024,Robert Jones,started games at guard at the junior college...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN,-1


In [10]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
embeddings = model.encode(profiles['draft_profile'].tolist())

In [12]:
embeddings

array([[-0.02400924,  0.02246189, -0.07962109, ...,  0.02992326,
         0.02146961,  0.0603883 ],
       [-0.04543393, -0.03509943, -0.02980237, ..., -0.04981454,
         0.00856797,  0.05311377],
       [-0.03942119, -0.06392992, -0.02087859, ..., -0.01501579,
        -0.01959313,  0.05965219],
       ...,
       [ 0.00376847, -0.01687002, -0.03105178, ..., -0.02508315,
         0.02738197,  0.01849411],
       [-0.01264568,  0.06112486, -0.10176325, ..., -0.0717894 ,
        -0.03378281,  0.06250897],
       [-0.0297422 ,  0.04647798, -0.07648223, ..., -0.01655806,
        -0.00338332,  0.05657864]], dtype=float32)

In [13]:
similarity_matrix = cosine_similarity(embeddings)








In [14]:
similarity_df = pd.DataFrame(similarity_matrix, index=profiles['player_name'], columns=profiles['player_name'])

In [15]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,1.000000,0.476609,0.565490,0.630389,0.693408,0.665695,0.635621,0.598615,0.744405,0.719425,...,0.421170,0.498646,0.490084,0.522450,0.566552,0.639733,0.452651,0.411738,0.608421,0.517027
Ronnie Brown,0.476609,1.000000,0.551036,0.593315,0.614389,0.628726,0.559088,0.488225,0.698091,0.634652,...,0.360399,0.417056,0.409443,0.463884,0.483756,0.494834,0.443447,0.402822,0.487059,0.436254
Braylon Edwards,0.565490,0.551036,1.000000,0.582401,0.509050,0.583387,0.636203,0.454950,0.610896,0.543281,...,0.378306,0.388655,0.451020,0.472596,0.462012,0.527899,0.401555,0.308166,0.451736,0.396572
Cedric Benson,0.630389,0.593315,0.582401,1.000000,0.734261,0.683382,0.639787,0.560410,0.727590,0.663984,...,0.391923,0.608510,0.449053,0.597404,0.537800,0.616868,0.649721,0.460028,0.637431,0.649840
Carnell Williams,0.693408,0.614389,0.509050,0.734261,1.000000,0.665222,0.682911,0.585587,0.755781,0.746248,...,0.458924,0.557263,0.477142,0.591990,0.614715,0.668942,0.621459,0.513425,0.720302,0.678442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.639733,0.494834,0.527899,0.616868,0.668942,0.751720,0.538485,0.620077,0.665900,0.577951,...,0.575520,0.606856,0.531062,0.678947,0.747179,1.000000,0.689785,0.550854,0.674375,0.678319
Dax Milne,0.452651,0.443447,0.401555,0.649721,0.621459,0.539934,0.478028,0.477960,0.547221,0.540237,...,0.373212,0.700469,0.364211,0.659687,0.691605,0.689785,1.000000,0.540840,0.688651,0.841563
Grant Stuard,0.411738,0.402822,0.308166,0.460028,0.513425,0.489156,0.332166,0.428776,0.445484,0.422010,...,0.282819,0.513424,0.330787,0.464280,0.510165,0.550854,0.540840,1.000000,0.512719,0.552782


In [16]:
similarity_df.sort_values(by = 'Joe Burrow', ascending = False).head(10)


player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Joe Burrow,0.667249,0.547796,0.561737,0.642604,0.612085,0.721309,0.553823,0.599182,0.695061,0.665384,...,0.489352,0.651098,0.530777,0.719868,0.648994,0.747928,0.646367,0.491220,0.634873,0.641745
Ian Book,0.677213,0.508692,0.522642,0.633699,0.648859,0.695707,0.520154,0.588289,0.683958,0.632478,...,0.482333,0.659866,0.539467,0.633693,0.722461,0.778007,0.675836,0.550637,0.677245,0.672414
Nate Stanley,0.737348,0.614020,0.576452,0.733018,0.699328,0.768714,0.603242,0.599311,0.812583,0.753900,...,0.466411,0.643726,0.562509,0.707718,0.670819,0.731490,0.615847,0.511944,0.680935,0.609126
Trevor Lawrence,0.671830,0.505358,0.537787,0.717629,0.668754,0.718560,0.565086,0.563842,0.672952,0.668257,...,0.445431,0.677671,0.541324,0.677173,0.656242,0.742115,0.713510,0.551294,0.666832,0.735148
Drew Lock,0.751464,0.529165,0.541071,0.675271,0.710035,0.729464,0.589260,0.610645,0.742780,0.700076,...,0.439508,0.593657,0.543229,0.690778,0.652797,0.732997,0.599099,0.461326,0.671050,0.645262
D'Andre Walker,0.652270,0.594286,0.535224,0.739839,0.694255,0.746739,0.583279,0.586571,0.731138,0.719525,...,0.505675,0.691630,0.487579,0.746534,0.649694,0.729423,0.690127,0.546115,0.713333,0.711434
Dwayne Haskins,0.695432,0.665463,0.587155,0.737369,0.719417,0.741255,0.625397,0.593705,0.814710,0.751444,...,0.473878,0.632372,0.523410,0.733306,0.708663,0.739624,0.697340,0.509698,0.685552,0.666482
Zach Wilson,0.707770,0.557548,0.561886,0.694530,0.659721,0.721096,0.582947,0.597711,0.721088,0.684349,...,0.420093,0.615476,0.491567,0.651805,0.634174,0.712509,0.635601,0.501284,0.648547,0.639673
Cole McDonald,0.702535,0.585806,0.594603,0.699545,0.687694,0.672007,0.630501,0.539194,0.745097,0.717434,...,0.453400,0.569191,0.503541,0.674256,0.516445,0.657116,0.590258,0.454833,0.655197,0.606443


In [17]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,1.000000,0.476609,0.565490,0.630389,0.693408,0.665695,0.635621,0.598615,0.744405,0.719425,...,0.421170,0.498646,0.490084,0.522450,0.566552,0.639733,0.452651,0.411738,0.608421,0.517027
Ronnie Brown,0.476609,1.000000,0.551036,0.593315,0.614389,0.628726,0.559088,0.488225,0.698091,0.634652,...,0.360399,0.417056,0.409443,0.463884,0.483756,0.494834,0.443447,0.402822,0.487059,0.436254
Braylon Edwards,0.565490,0.551036,1.000000,0.582401,0.509050,0.583387,0.636203,0.454950,0.610896,0.543281,...,0.378306,0.388655,0.451020,0.472596,0.462012,0.527899,0.401555,0.308166,0.451736,0.396572
Cedric Benson,0.630389,0.593315,0.582401,1.000000,0.734261,0.683382,0.639787,0.560410,0.727590,0.663984,...,0.391923,0.608510,0.449053,0.597404,0.537800,0.616868,0.649721,0.460028,0.637431,0.649840
Carnell Williams,0.693408,0.614389,0.509050,0.734261,1.000000,0.665222,0.682911,0.585587,0.755781,0.746248,...,0.458924,0.557263,0.477142,0.591990,0.614715,0.668942,0.621459,0.513425,0.720302,0.678442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.639733,0.494834,0.527899,0.616868,0.668942,0.751720,0.538485,0.620077,0.665900,0.577951,...,0.575520,0.606856,0.531062,0.678947,0.747179,1.000000,0.689785,0.550854,0.674375,0.678319
Dax Milne,0.452651,0.443447,0.401555,0.649721,0.621459,0.539934,0.478028,0.477960,0.547221,0.540237,...,0.373212,0.700469,0.364211,0.659687,0.691605,0.689785,1.000000,0.540840,0.688651,0.841563
Grant Stuard,0.411738,0.402822,0.308166,0.460028,0.513425,0.489156,0.332166,0.428776,0.445484,0.422010,...,0.282819,0.513424,0.330787,0.464280,0.510165,0.550854,0.540840,1.000000,0.512719,0.552782


In [18]:
joe_shiesty = similarity_df.loc['Joe Burrow']

In [19]:
joe_shiesty.sort_values(ascending = False).head(20)

,Joe Burrow
player_name,
Joe Burrow,1.000000
Ian Book,0.862507
Nate Stanley,0.860192
Trevor Lawrence,0.857085
Drew Lock,0.852390
D'Andre Walker,0.845148
Dwayne Haskins,0.843203
Zach Wilson,0.842394
Cole McDonald,0.825624


In [20]:
# generate the three most similar players to Joe Burrow, making sure they match his position and were drafted in the same round

